In [7]:
!pip install optuna optuna-integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 976.2 kB/s eta 0:00:00 eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 777.4 kB/s eta 0:00:00B/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 488.5 kB/s eta 0:00:00m eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 118.5 kB/s eta 0:00:001m127.1 kB/s eta 0:00:01


In [6]:
import torch
def get_device():
  device = "cpu"
  if (torch.cuda.is_available()):
    num_gpus = torch.cuda.device_count()
    device = [-1] * num_gpus
  return device
get_device()

'cpu'

In [ ]:
import optuna
import os
models = ["yolo12n", "yolo12s", "yolo12m", "yolo12l", "yolo12x"]
data = "./data/data.yaml"
epochs= 10000
patience= 100
min_batch_size= 1
max_batch_size= 256
batch_size_step = 16
image_size = [244, 480, 640, 800, 1024]
cache = True
optimizer = ["SGD", "Adam", "AdamW", "NAdam", "RAdam", "RMSProp"]
multi_scale = [True, False]
cos_lr = [True, False]
close_mosaic_min = 0
close_mosaic_max = 100
amp = True
lr0_min = 1e-5
lr0_max = 1e-1
momentum_min = 0.5
momentum_max = 0.99
weight_decay_min = 1e-6
weight_decay_max = 1e-2
warmup_epochs_min = 0
warmup_epochs_max = 100
warmup_momentum_min = 0.5
warmup_momentum_max = 0.99
warmup_bias_lr_min = 0.0
warmup_bias_lr_max = 0.2
box_weight_min = 0.0
box_weight_max = 10.0
cls_weight_min = 0.0
cls_weight_max = 10.0
dfl_weight_min = 0.0
dfl_weight_max = 10.0
dropout_min = 0.0
dropout_max = 0.5
max_workers = max(1, (len(os.sched_getaffinity(0)) if hasattr(os, 'sched_getaffinity') else os.cpu_count()) - 1)

Using max_workers = 3


In [16]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 760.5 kB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 455.5 kB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:01


In [ ]:
import torch
import gc
from ultralytics import YOLO

def train(trial):
    try:
        # Model selection
        model_name = trial.suggest_categorical("model", models)
        
        # Basic training parameters
        batch_size = trial.suggest_int("batch_size", min_batch_size, max_batch_size, step=batch_size_step)
        imgsz = trial.suggest_categorical("imgsz", image_size)
        
        # Optimizer
        optimizer_name = trial.suggest_categorical("optimizer", optimizer)
        
        # Training settings
        multi_scale_enabled = trial.suggest_categorical("multi_scale", multi_scale)
        cos_lr_enabled = trial.suggest_categorical("cos_lr", cos_lr)
        close_mosaic = trial.suggest_int("close_mosaic", close_mosaic_min, close_mosaic_max)
        
        # Learning rate parameters
        lr0 = trial.suggest_float("lr0", lr0_min, lr0_max, log=True)
        momentum = trial.suggest_float("momentum", momentum_min, momentum_max)
        weight_decay = trial.suggest_float("weight_decay", weight_decay_min, weight_decay_max, log=True)
        
        # Warm-up parameters
        warmup_epochs = trial.suggest_int("warmup_epochs", warmup_epochs_min, warmup_epochs_max)
        warmup_momentum = trial.suggest_float("warmup_momentum", warmup_momentum_min, warmup_momentum_max)
        warmup_bias_lr = trial.suggest_float("warmup_bias_lr", warmup_bias_lr_min, warmup_bias_lr_max)
        
        # Loss function weights
        box_weight = trial.suggest_float("box_weight", box_weight_min, box_weight_max)
        cls_weight = trial.suggest_float("cls_weight", cls_weight_min, cls_weight_max)
        dfl_weight = trial.suggest_float("dfl_weight", dfl_weight_min, dfl_weight_max)
        
        # Dropout
        dropout = trial.suggest_float("dropout", dropout_min, dropout_max)
        
        print(f"Trial {trial.number} parameters:")
        print(f"   Model: {model_name}, Batch: {batch_size}, ImgSz: {imgsz}")
        print(f"   Optimizer: {optimizer_name}, LR: {lr0:.6f}, Momentum: {momentum:.3f}")
        print("-" * 60)
    
        model = YOLO(f"{model_name}.pt")
        
        results = model.train(
            data=data,
            epochs=epochs,
            patience=patience,
            batch=batch_size,
            imgsz=imgsz,
            cache=cache,
            optimizer=optimizer_name,
            multi_scale=multi_scale_enabled,
            cos_lr=cos_lr_enabled,
            close_mosaic=close_mosaic,
            amp=amp,
            lr0=lr0,
            momentum=momentum,
            weight_decay=weight_decay,
            warmup_epochs=warmup_epochs,
            warmup_momentum=warmup_momentum,
            warmup_bias_lr=warmup_bias_lr,
            box=box_weight,
            cls=cls_weight,
            dfl=dfl_weight,
            dropout=dropout,
            verbose=False,
            save=False,
            plots=False,
            device=get_device(),
            workers=max_workers
        )
        
        mAP = results.metrics.box.map50_95
        print(f"Trial {trial.number} completed: mAP = {mAP:.4f}")
        
        return mAP
        
    except torch.cuda.OutOfMemoryError as e:
        print(f"Trial {trial.number} - CUDA Out of Memory Error!")
        print(f"   Model: {model_name}, Batch: {batch_size}, ImgSz: {imgsz}")
        print(f"   Erro: {str(e)}")
        
        if 'model' in locals():
            del model
        torch.cuda.empty_cache()
        gc.collect()
        return None
        
    except Exception as e:
        print(f"Trial {trial.number} - Error: {type(e).__name__}")
        print(f"   Detalhes: {str(e)}")
        
        if 'model' in locals():
            del model
        torch.cuda.empty_cache()
        gc.collect()
        
        return None
        
    finally:
        if 'model' in locals():
            del model
        torch.cuda.empty_cache()
        gc.collect()

In [3]:
import optuna
from optuna.pruners import BasePruner
from optuna.trial import TrialState

class ErrorPruner(BasePruner):
    def __init__(self, max_consecutive_errors=3):
        self.max_consecutive_errors = max_consecutive_errors
        self.error_count = 0

    def prune(self, study, trial):
        if trial.state == TrialState.FAIL:
            self.error_count += 1
            print(f"Trial {trial.number} failed. Consecutive errors: {self.error_count}")
            if self.error_count >= self.max_consecutive_errors:
                print(f"Many consecutive errors ({self.error_count}). Consider reviewing configuration.")
            return True
        else:
            self.error_count = 0
            return False

In [ ]:
def optimize_hyperparameters(n_trials):
    study = optuna.create_study(direction="minimize",
                                study_name="YOLO_Hyperparameter_Optimization",
                                storage="sqlite:///yolo_hyperparameter_optimization.db",
                                load_if_exists=False,
                                pruner=ErrorPruner(max_consecutive_errors=5)
                                )
    study.optimize(train, n_trials=n_trials)
    
    print("Best hyperparameters found:")
    print("=" * 50)
    for key, value in study.best_params.items():
        print(f"{key}: {value}")
    
    print(f"\nBest mAP50-95: {study.best_value:.4f}")
    
    return study

In [ ]:
study = optimize_hyperparameters(n_trials=50)

import optuna.visualization as vis

fig = vis.plot_optimization_history(study)
fig.show()

fig = vis.plot_param_importances(study)
fig.show()

print("\nTraining final model with best parameters...")
best_params = study.best_params

final_model = YOLO(f"{best_params['model']}.pt")
final_results = final_model.train(
    data=data,
    epochs=epochs,
    patience=patience,
    batch=best_params['batch_size'],
    imgsz=best_params['imgsz'],
    cache=cache,
    optimizer=best_params['optimizer'],
    multi_scale=best_params['multi_scale'],
    cos_lr=best_params['cos_lr'],
    close_mosaic=best_params['close_mosaic'],
    amp=amp,
    lr0=best_params['lr0'],
    momentum=best_params['momentum'],
    weight_decay=best_params['weight_decay'],
    warmup_epochs=best_params['warmup_epochs'],
    warmup_momentum=best_params['warmup_momentum'],
    warmup_bias_lr=best_params['warmup_bias_lr'],
    box=best_params['box_weight'],
    cls=best_params['cls_weight'],
    dfl=best_params['dfl_weight'],
    dropout=best_params['dropout'],
    name='best_model',
    save=True,
    plots=True
)

print("Optimization completed!")

[I 2025-10-31 23:08:39,972] A new study created in RDB with name: YOLO_Hyperparameter_Optimization


Ultralytics 8.3.223 🚀 Python-3.12.2 torch-2.8.0+cpu CPU (Intel Core(TM) i7-5500U 2.40GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=97, bgr=0.0, box=8.378851255441765, cache=True, cfg=None, classes=None, close_mosaic=44, cls=7.512275430707898, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=6.248166613630108, dnn=False, dropout=0.14864161765082806, dynamic=False, embed=None, epochs=10000, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.010556784490723264, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.8834463556290939, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, o